# Databricks notebook source

 # Simplified Mining Operations Pipeline
 
 This Lakeflow Declarative Pipelines pipeline processes simplified mining operations data through a medallion architecture:
 
 ## Pipeline Architecture:
 - **Bronze Layer**: Raw data ingestion from Unity Catalog volume (Streaming Tables)
 - **Silver Layer**: Cleaned and enriched data with business logic (Materialized Views)
 - **Gold Layer**: Aggregated and analytical tables for reporting (Materialized Views)
 
 ## Core Datasets:
 1. **Truck Productivity Events**: Operational events with delays and breakdowns (80% productive)
 2. **Truck Cycles**: Loading and dumping transactions with source/destination and timing
 3. **Time Usage Model**: Event categorization for operational efficiency analysis
 
 ## Data Flow:
 1. **Bronze Tables**: Load raw parquet files from `/Volumes/mining_operations/production/raw_data/` as streaming tables
 2. **Silver Views**: Apply data quality checks, transformations, and business rules as materialized views
 3. **Gold Views**: Create analytical views and aggregations for operational efficiency insights as materialized views

 ## Bronze Layer - Raw Data Ingestion (Streaming Tables)
 
 Load raw data from Unity Catalog volume with minimal transformations using streaming tables for append-only processing.

In [ ]:
 -- Bronze: Raw Trucks Data (Streaming Table)
 CREATE OR REFRESH STREAMING TABLE bronze_trucks
 COMMENT "Raw trucks data from Unity Catalog volume - streaming table for append-only processing"
 AS SELECT 
   truck_id,
   truck_type,
   capacity_tonnes,
   created_at,
   current_timestamp() as bronze_loaded_at
 FROM cloud_files(
   "/Volumes/mining_operations/production/raw_data/trucks",
   "parquet"
 );

In [ ]:
 -- Bronze: Raw Truck Cycles Data (Streaming Table)
 CREATE OR REFRESH STREAMING TABLE bronze_truck_cycles
 COMMENT "Raw truck cycles data for loading and dumping events - streaming table for append-only processing"
 AS SELECT 
   cycle_id,
   truck_id,
   payload_tonnes,
   loading_start_time,
   loading_end_time,
   travel_start_time,
   travel_end_time,
   dumping_start_time,
   dumping_end_time,
   total_cycle_duration_minutes,
   cycle_status,
   created_at,
   current_timestamp() as bronze_loaded_at
 FROM cloud_files(
   "/Volumes/mining_operations/production/raw_data/truck_cycles",
   "parquet"
 );

In [ ]:
 -- Bronze: Raw Truck Productivity Events Data (Streaming Table)
 CREATE OR REFRESH STREAMING TABLE bronze_truck_productivity_events
 COMMENT "Raw truck productivity events data with delays and breakdowns - streaming table for append-only processing"
 AS SELECT 
   event_id,
   truck_id,
   event_timestamp,
   event_type,
   event_category,
   duration_minutes,
   event_details,
   location_x,
   location_y,
   created_at,
   current_timestamp() as bronze_loaded_at
 FROM cloud_files(
   "/Volumes/mining_operations/production/raw_data/truck_productivity_events",
   "parquet"
 );

In [ ]:
 -- Bronze: Raw Time Usage Model Data (Streaming Table)
 CREATE OR REFRESH STREAMING TABLE bronze_time_usage_model
 COMMENT "Raw time usage model data for categorizing productivity events - streaming table for append-only processing"
 AS SELECT 
   event_type,
   category,
   subcategory,
   description,
   target_efficiency_percentage,
   is_productive,
   created_at,
   current_timestamp() as bronze_loaded_at
 FROM cloud_files(
   "/Volumes/mining_operations/production/raw_data/time_usage_model",
   "parquet"
 );

 ## Silver Layer - Cleaned and Enriched Data (Materialized Views)
 
 Apply data quality checks, transformations, and business logic to create clean, reliable datasets using materialized views for efficient processing.

In [ ]:
 -- Silver: Cleaned Trucks Data with Quality Checks (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW silver_trucks(
   CONSTRAINT valid_capacity EXPECT (capacity_tonnes BETWEEN 280 AND 500),
   CONSTRAINT not_null_truck_id EXPECT (truck_id IS NOT NULL),
   CONSTRAINT not_null_truck_type EXPECT (truck_type IS NOT NULL)
 )
 COMMENT "Cleaned trucks data with data quality validations - materialized view for efficient processing"
 TBLPROPERTIES ("quality" = "silver")
 AS SELECT 
   truck_id,
   truck_type,
   capacity_tonnes,
   created_at,
   bronze_loaded_at,
   current_timestamp() as silver_processed_at,
   -- Data quality flags
   CASE 
     WHEN capacity_tonnes BETWEEN 280 AND 500 THEN 'Valid'
     ELSE 'Invalid'
   END as capacity_quality_flag
 FROM bronze_trucks
 WHERE truck_id IS NOT NULL
   AND truck_type IS NOT NULL
   AND capacity_tonnes IS NOT NULL;

In [ ]:
 -- Silver: Cleaned Truck Cycles Data (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW silver_truck_cycles(
   CONSTRAINT valid_payload EXPECT (payload_tonnes BETWEEN 200 AND 600),
   CONSTRAINT valid_cycle_duration EXPECT (total_cycle_duration_minutes BETWEEN 20 AND 120),
   CONSTRAINT not_null_cycle_id EXPECT (cycle_id IS NOT NULL),
   CONSTRAINT not_null_truck_id EXPECT (truck_id IS NOT NULL)
 )
 COMMENT "Cleaned truck cycles data with performance calculations - materialized view for efficient processing"
 TBLPROPERTIES ("quality" = "silver")
 AS SELECT 
   cycle_id,
   truck_id,
   payload_tonnes,
   loading_start_time,
   loading_end_time,
   travel_start_time,
   travel_end_time,
   dumping_start_time,
   dumping_end_time,
   total_cycle_duration_minutes,
   cycle_status,
   created_at,
   bronze_loaded_at,
   current_timestamp() as silver_processed_at,
   -- Convert timestamps
   CAST(loading_start_time AS TIMESTAMP) as loading_start_ts,
   CAST(loading_end_time AS TIMESTAMP) as loading_end_ts,
   CAST(travel_start_time AS TIMESTAMP) as travel_start_ts,
   CAST(travel_end_time AS TIMESTAMP) as travel_end_ts,
   CAST(dumping_start_time AS TIMESTAMP) as dumping_start_ts,
   CAST(dumping_end_time AS TIMESTAMP) as dumping_end_ts
 FROM bronze_truck_cycles
 WHERE cycle_id IS NOT NULL
   AND truck_id IS NOT NULL
   AND payload_tonnes IS NOT NULL;

In [ ]:
 -- Silver: Cleaned Truck Productivity Events Data (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW silver_truck_productivity_events(
   CONSTRAINT valid_duration EXPECT (duration_minutes BETWEEN 0 AND 1440),
   CONSTRAINT not_null_event_id EXPECT (event_id IS NOT NULL),
   CONSTRAINT not_null_truck_id EXPECT (truck_id IS NOT NULL),
   CONSTRAINT not_null_event_category EXPECT (event_category IS NOT NULL)
 )
 COMMENT "Cleaned truck productivity events data with quality validations - materialized view for efficient processing"
 TBLPROPERTIES ("quality" = "silver")
 AS SELECT 
   event_id,
   truck_id,
   CAST(event_timestamp AS TIMESTAMP) as event_timestamp_ts,
   event_type,
   event_category,
   duration_minutes,
   event_details,
   location_x,
   location_y,
   created_at,
   bronze_loaded_at,
   current_timestamp() as silver_processed_at,
   -- Quality flags
   CASE 
     WHEN duration_minutes BETWEEN 0 AND 1440 THEN 'Valid'
     ELSE 'Invalid'
   END as duration_quality_flag
 FROM bronze_truck_productivity_events
 WHERE event_id IS NOT NULL
   AND truck_id IS NOT NULL
   AND event_timestamp IS NOT NULL
   AND event_category IS NOT NULL;

In [ ]:
 -- Silver: Cleaned Time Usage Model Data (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW silver_time_usage_model(
   CONSTRAINT not_null_event_type EXPECT (event_type IS NOT NULL),
   CONSTRAINT not_null_category EXPECT (category IS NOT NULL),
   CONSTRAINT valid_efficiency EXPECT (target_efficiency_percentage BETWEEN 0 AND 100),
   CONSTRAINT valid_productive_flag EXPECT (is_productive IN (true, false))
 )
 COMMENT "Cleaned time usage model data with data quality validations - materialized view for efficient processing"
 TBLPROPERTIES ("quality" = "silver")
 AS SELECT
   event_type,
   category,
   subcategory,
   description,
   target_efficiency_percentage,
   is_productive,
   created_at,
   bronze_loaded_at,
   current_timestamp() as silver_processed_at,
   -- Quality flags
   CASE 
     WHEN target_efficiency_percentage BETWEEN 0 AND 100 THEN 'Valid'
     ELSE 'Invalid'
   END as efficiency_quality_flag,
   CASE 
     WHEN is_productive IN (true, false) THEN 'Valid'
     ELSE 'Invalid'
   END as productive_flag_quality
 FROM bronze_time_usage_model
 WHERE event_type IS NOT NULL
   AND category IS NOT NULL
   AND target_efficiency_percentage IS NOT NULL
   AND is_productive IS NOT NULL;

 ## Gold Layer - Operational Efficiency Analytics (Materialized Views)
 
 Create focused analytical tables for operational efficiency reporting and insights using materialized views for pre-computed aggregations.

In [ ]:
 -- Gold: Daily Operational Efficiency Summary (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW gold_daily_metrics
 COMMENT "Daily operational efficiency metrics - materialized view for pre-computed aggregations"
 TBLPROPERTIES ("quality" = "gold")
 AS SELECT 
   DATE(event_timestamp_ts) as operation_date,
   COUNT(DISTINCT truck_id) as active_trucks,
   COUNT(*) as total_events,
   SUM(duration_minutes) as total_operational_minutes,
   ROUND(
     (SUM(CASE WHEN event_category = 'Productive' THEN duration_minutes ELSE 0 END) * 100.0) / 
     NULLIF(SUM(duration_minutes), 0), 2
   ) as productive_time_percentage,
   ROUND(
     (SUM(CASE WHEN event_category = 'Breakdown' THEN duration_minutes ELSE 0 END) * 100.0) / 
     NULLIF(SUM(duration_minutes), 0), 2
   ) as breakdown_time_percentage,
   ROUND(
     (SUM(CASE WHEN event_category = 'Delay' THEN duration_minutes ELSE 0 END) * 100.0) / 
     NULLIF(SUM(duration_minutes), 0), 2
   ) as delay_time_percentage,
   current_timestamp() as gold_processed_at
 FROM silver_truck_productivity_events
 WHERE event_timestamp_ts IS NOT NULL
 GROUP BY DATE(event_timestamp_ts);

In [ ]:
 -- Gold: Events with Time Usage Analysis (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW gold_raw_events_with_time_usage
 COMMENT "Comprehensive events dataset with time usage categorization and efficiency metrics - materialized view for detailed analysis"
 TBLPROPERTIES ("quality" = "gold")
 AS SELECT 
   pe.event_id,
   pe.truck_id,
   pe.event_timestamp_ts,
   pe.event_type,
   pe.event_category,
   pe.duration_minutes,
   pe.event_details,
   pe.location_x,
   pe.location_y,
   pe.created_at,
   pe.silver_processed_at,
   -- Time Usage Model Fields
   tum.category as time_usage_category,
   tum.subcategory as time_usage_subcategory,
   tum.description as time_usage_description,
   tum.target_efficiency_percentage,
   tum.is_productive,
   -- Calculated Efficiency Metrics
   CASE 
     WHEN tum.is_productive = true THEN 'Productive'
     ELSE 'Non-Productive'
   END as productivity_classification,
   ROUND(
     (pe.duration_minutes * tum.target_efficiency_percentage) / 100.0, 2
   ) as effective_productive_minutes,
   ROUND(
     pe.duration_minutes - ((pe.duration_minutes * tum.target_efficiency_percentage) / 100.0), 2
   ) as lost_productivity_minutes,
   -- Efficiency Rating
   CASE 
     WHEN tum.target_efficiency_percentage >= 90 THEN 'Excellent'
     WHEN tum.target_efficiency_percentage >= 75 THEN 'Good'
     WHEN tum.target_efficiency_percentage >= 50 THEN 'Fair'
     WHEN tum.target_efficiency_percentage >= 25 THEN 'Poor'
     ELSE 'Critical'
   END as efficiency_rating,
   -- Time of Day Analysis
   HOUR(pe.event_timestamp_ts) as event_hour,
   CASE 
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 6 AND 11 THEN 'Morning'
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 12 AND 17 THEN 'Afternoon'
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 18 AND 23 THEN 'Evening'
     ELSE 'Night'
   END as time_of_day,
   -- Day of Week Analysis
   DAYOFWEEK(pe.event_timestamp_ts) as day_of_week,
   CASE 
     WHEN DAYOFWEEK(pe.event_timestamp_ts) IN (1, 7) THEN 'Weekend'
     ELSE 'Weekday'
   END as day_type,
   current_timestamp() as gold_processed_at
 FROM silver_truck_productivity_events pe
 LEFT JOIN silver_time_usage_model tum ON pe.event_type = tum.event_type
 WHERE pe.event_timestamp_ts IS NOT NULL
   AND tum.event_type IS NOT NULL
 ORDER BY pe.event_timestamp_ts DESC, pe.truck_id;


In [ ]:
 -- Gold: Time Usage Efficiency Summary (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW gold_truck_time_usage_by_day
 COMMENT "Aggregated time usage efficiency metrics by category, truck, and time periods - materialized view for executive reporting"
 TBLPROPERTIES ("quality" = "gold")
 AS SELECT 
   -- Grouping Dimensions
   DATE(pe.event_timestamp_ts) as event_date,
   pe.truck_id,
   tum.category as time_usage_category,
   tum.subcategory as time_usage_subcategory,
   CASE 
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 6 AND 11 THEN 'Morning'
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 12 AND 17 THEN 'Afternoon'
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 18 AND 23 THEN 'Evening'
     ELSE 'Night'
   END as time_of_day,
   CASE 
     WHEN DAYOFWEEK(pe.event_timestamp_ts) IN (1, 7) THEN 'Weekend'
     ELSE 'Weekday'
   END as day_type,
   -- Aggregated Metrics
   COUNT(*) as total_events,
   SUM(pe.duration_minutes) as total_duration_minutes,
   AVG(pe.duration_minutes) as avg_duration_minutes,
   SUM(pe.effective_productive_minutes) as total_effective_productive_minutes,
   SUM(pe.lost_productivity_minutes) as total_lost_productivity_minutes,
   -- Efficiency Calculations
   ROUND(
     (SUM(pe.effective_productive_minutes) / NULLIF(SUM(pe.duration_minutes), 0)) * 100, 2
   ) as overall_efficiency_percentage,
   ROUND(
     (SUM(CASE WHEN tum.is_productive = true THEN pe.duration_minutes ELSE 0 END) / 
      NULLIF(SUM(pe.duration_minutes), 0)) * 100, 2
   ) as productive_time_percentage,
   ROUND(
     (SUM(CASE WHEN tum.is_productive = false THEN pe.duration_minutes ELSE 0 END) / 
      NULLIF(SUM(pe.duration_minutes), 0)) * 100, 2
   ) as non_productive_time_percentage,
   -- Efficiency Rating Distribution
   SUM(CASE WHEN pe.efficiency_rating = 'Excellent' THEN 1 ELSE 0 END) as excellent_events,
   SUM(CASE WHEN pe.efficiency_rating = 'Good' THEN 1 ELSE 0 END) as good_events,
   SUM(CASE WHEN pe.efficiency_rating = 'Fair' THEN 1 ELSE 0 END) as fair_events,
   SUM(CASE WHEN pe.efficiency_rating = 'Poor' THEN 1 ELSE 0 END) as poor_events,
   SUM(CASE WHEN pe.efficiency_rating = 'Critical' THEN 1 ELSE 0 END) as critical_events,
   current_timestamp() as gold_processed_at
 FROM gold_events_with_time_usage pe
 LEFT JOIN silver_time_usage_model tum ON pe.event_type = tum.event_type
 WHERE pe.event_timestamp_ts IS NOT NULL
 GROUP BY 
   DATE(pe.event_timestamp_ts),
   pe.truck_id,
   tum.category,
   tum.subcategory,
   CASE 
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 6 AND 11 THEN 'Morning'
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 12 AND 17 THEN 'Afternoon'
     WHEN HOUR(pe.event_timestamp_ts) BETWEEN 18 AND 23 THEN 'Evening'
     ELSE 'Night'
   END,
   CASE 
     WHEN DAYOFWEEK(pe.event_timestamp_ts) IN (1, 7) THEN 'Weekend'
     ELSE 'Weekday'
   END
 ORDER BY event_date DESC, pe.truck_id, total_duration_minutes DESC;


In [ ]:
 -- Gold: Weekly Operational Efficiency Metrics by Category (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW gold_weekly_metrics
 COMMENT "Operational efficiency metrics by event category - materialized view for pre-computed aggregations"
 TBLPROPERTIES ("quality" = "gold")
 AS SELECT 
   DATE_TRUNC('WEEK', pe.event_timestamp_ts) as week_start_date,
   pe.event_category,
   pe.event_type,
   COUNT(*) as event_count,
   SUM(pe.duration_minutes) as total_duration_minutes,
   AVG(pe.duration_minutes) as avg_duration_minutes,
   COUNT(DISTINCT pe.truck_id) as affected_trucks,
   ROUND(
     (SUM(CASE WHEN pe.event_category = 'Productive' THEN pe.duration_minutes ELSE 0 END) * 100.0) / 
     NULLIF(SUM(pe.duration_minutes), 0), 2
   ) as productive_time_percentage,
   ROUND(
     (SUM(CASE WHEN pe.event_category = 'Breakdown' THEN pe.duration_minutes ELSE 0 END) * 100.0) / 
     NULLIF(SUM(pe.duration_minutes), 0), 2
   ) as breakdown_time_percentage,
   ROUND(
     (SUM(CASE WHEN pe.event_category = 'Delay' THEN pe.duration_minutes ELSE 0 END) * 100.0) / 
     NULLIF(SUM(pe.duration_minutes), 0), 2
   ) as delay_time_percentage,
   current_timestamp() as gold_processed_at
 FROM silver_truck_productivity_events pe
 WHERE pe.event_category IS NOT NULL
   AND pe.event_timestamp_ts IS NOT NULL
 GROUP BY DATE_TRUNC('WEEK', pe.event_timestamp_ts), pe.event_category, pe.event_type
 ORDER BY week_start_date DESC, pe.event_category, pe.event_type;

 ## Pipeline Summary
 
 This simplified Lakeflow Declarative Pipelines pipeline creates a focused data processing architecture for mining operations efficiency analysis:
 
 ### **Bronze Layer (4 Streaming Tables)**
 - Raw data ingestion from Unity Catalog volume
 - **Streaming Tables**: Each record processed exactly once for append-only sources
 - Minimal transformations, preserves original data
 - Core datasets: Trucks, Truck Cycles, Truck Productivity Events, Time Usage Model
 
 ### **Silver Layer (4 Materialized Views)**
 - Data quality checks and validations
 - Business logic and calculated fields
 - Time usage model with quality flags
 - **Materialized Views**: Efficient processing for transformations and aggregations
 - **Data Quality Constraints**: Built-in EXPECT constraints for data validation
 
 ### **Gold Layer (7 Materialized Views)**
 - **Daily operational efficiency summary**: Daily aggregated metrics
 - **Truck performance analytics**: Individual truck performance analysis
 - **Operational efficiency metrics by category**: Category-based efficiency analysis
 - **Weekly operational efficiency metrics**: Weekly category-based efficiency analysis
 - **Events with time usage analysis**: Comprehensive event dataset with time usage mapping
 - **Time usage efficiency summary**: Aggregated efficiency metrics by category, truck, and time periods
 - **Truck cycles with truck details**: Comprehensive cycle analysis with truck specifications and performance metrics
 - **Materialized Views**: Pre-computed aggregations for fast query performance
 
 ### **Enhanced Time Usage Analysis Features**
 - **Event-Time Usage Mapping**: Each event mapped to its time usage categorization
 - **Efficiency Calculations**: Effective productive minutes vs lost productivity minutes
 - **Efficiency Ratings**: Excellent, Good, Fair, Poor, Critical classifications
 - **Time-based Analysis**: Morning, Afternoon, Evening, Night time periods
 - **Day Type Analysis**: Weekend vs Weekday performance patterns
 - **Comprehensive Metrics**: Productivity classification, efficiency percentages, and rating distributions
 
 ### **Key Gold Tables for Analysis**
 1. **`gold_events_with_time_usage`**: Detailed event-level data with time usage mapping
 2. **`gold_time_usage_efficiency_summary`**: Aggregated efficiency metrics for reporting
 3. **`gold_daily_operational_efficiency`**: Daily operational efficiency trends
 4. **`gold_truck_performance_by_week`**: Individual truck performance analysis
 5. **`gold_operational_efficiency_metrics`**: Category-based efficiency analysis
 6. **`gold_weekly_operation_efficiency_metrics`**: Weekly category-based efficiency analysis
 7. **`gold_truck_cycles`**: Comprehensive truck cycles with truck details and performance metrics
 
 The simplified pipeline focuses on core operational efficiency metrics with comprehensive time usage analysis and will automatically process new data as it arrives in the Unity Catalog volume.

In [ ]:
 -- Gold: Truck Cycles with Truck Details (Materialized View)
 CREATE OR REFRESH MATERIALIZED VIEW gold_truck_cycles
 COMMENT "Comprehensive truck cycles data with truck details and performance metrics - materialized view for detailed cycle analysis"
 TBLPROPERTIES ("quality" = "gold")
 AS SELECT 
   -- Cycle Information
   sc.cycle_id,
   sc.truck_id,
   sc.payload_tonnes,
   sc.loading_start_time,
   sc.loading_end_time,
   sc.travel_start_time,
   sc.travel_end_time,
   sc.dumping_start_time,
   sc.dumping_end_time,
   sc.total_cycle_duration_minutes,
   sc.cycle_status,
   sc.created_at,
   sc.bronze_loaded_at,
   sc.silver_processed_at,
   
   -- Converted Timestamps
   sc.loading_start_ts,
   sc.loading_end_ts,
   sc.travel_start_ts,
   sc.travel_end_ts,
   sc.dumping_start_ts,
   sc.dumping_end_ts,
   
   -- Truck Details
   st.truck_type,
   st.capacity_tonnes,
   st.created_at as truck_created_at,
   st.bronze_loaded_at as truck_bronze_loaded_at,
   st.silver_processed_at as truck_silver_processed_at,
   st.capacity_quality_flag,
   
   -- Calculated Performance Metrics
   ROUND((sc.payload_tonnes / NULLIF(st.capacity_tonnes, 0)) * 100, 2) as payload_utilization_percentage,
   ROUND(sc.total_cycle_duration_minutes / 60.0, 2) as total_cycle_duration_hours,
   
   -- Cycle Phase Durations
   ROUND((UNIX_TIMESTAMP(sc.loading_end_ts) - UNIX_TIMESTAMP(sc.loading_start_ts)) / 60.0, 2) as loading_duration_minutes,
   ROUND((UNIX_TIMESTAMP(sc.travel_end_ts) - UNIX_TIMESTAMP(sc.travel_start_ts)) / 60.0, 2) as travel_duration_minutes,
   ROUND((UNIX_TIMESTAMP(sc.dumping_end_ts) - UNIX_TIMESTAMP(sc.dumping_start_ts)) / 60.0, 2) as dumping_duration_minutes,
   
   -- Performance Indicators
   CASE 
     WHEN (sc.payload_tonnes / NULLIF(st.capacity_tonnes, 0)) >= 0.95 THEN 'Excellent'
     WHEN (sc.payload_tonnes / NULLIF(st.capacity_tonnes, 0)) >= 0.85 THEN 'Good'
     WHEN (sc.payload_tonnes / NULLIF(st.capacity_tonnes, 0)) >= 0.75 THEN 'Fair'
     ELSE 'Poor'
   END as payload_efficiency_rating,
   
   CASE 
     WHEN sc.total_cycle_duration_minutes <= 30 THEN 'Fast'
     WHEN sc.total_cycle_duration_minutes <= 60 THEN 'Normal'
     WHEN sc.total_cycle_duration_minutes <= 90 THEN 'Slow'
     ELSE 'Very Slow'
   END as cycle_speed_rating,
   
   -- Time-based Analysis
   DATE(sc.loading_start_ts) as cycle_date,
   HOUR(sc.loading_start_ts) as cycle_hour,
   DAYOFWEEK(sc.loading_start_ts) as day_of_week,
   CASE 
     WHEN HOUR(sc.loading_start_ts) BETWEEN 6 AND 11 THEN 'Morning'
     WHEN HOUR(sc.loading_start_ts) BETWEEN 12 AND 17 THEN 'Afternoon'
     WHEN HOUR(sc.loading_start_ts) BETWEEN 18 AND 23 THEN 'Evening'
     ELSE 'Night'
   END as time_of_day,
   CASE 
     WHEN DAYOFWEEK(sc.loading_start_ts) IN (1, 7) THEN 'Weekend'
     ELSE 'Weekday'
   END as day_type,
   
   current_timestamp() as gold_processed_at
   
 FROM silver_truck_cycles sc
 LEFT JOIN silver_trucks st ON sc.truck_id = st.truck_id
 WHERE sc.cycle_id IS NOT NULL
   AND sc.truck_id IS NOT NULL
   AND sc.payload_tonnes IS NOT NULL
   AND st.truck_id IS NOT NULL
 ORDER BY sc.loading_start_ts DESC, sc.truck_id;


In [ ]:
 -- Example Queries for the Gold Layer Tables
 -- 
 -- 1. Get all events with their time usage categorization and efficiency metrics
 -- SELECT * FROM gold_events_with_time_usage 
 -- WHERE event_date >= CURRENT_DATE - INTERVAL 7 DAYS
 -- ORDER BY event_timestamp_ts DESC;
 --
 -- 2. Find trucks with the highest productivity
 -- SELECT truck_id, 
 --        SUM(total_effective_productive_minutes) as total_productive_minutes,
 --        AVG(overall_efficiency_percentage) as avg_efficiency
 -- FROM gold_time_usage_efficiency_summary 
 -- WHERE event_date >= CURRENT_DATE - INTERVAL 7 DAYS
 -- GROUP BY truck_id 
 -- ORDER BY total_productive_minutes DESC;
 --
 -- 3. Analyze efficiency by time of day
 -- SELECT time_of_day,
 --        AVG(overall_efficiency_percentage) as avg_efficiency,
 --        SUM(total_duration_minutes) as total_minutes
 -- FROM gold_time_usage_efficiency_summary 
 -- WHERE event_date >= CURRENT_DATE - INTERVAL 7 DAYS
 -- GROUP BY time_of_day 
 -- ORDER BY avg_efficiency DESC;
 --
 -- 4. Find events with critical efficiency ratings
 -- SELECT truck_id, event_type, duration_minutes, 
 --        efficiency_rating, lost_productivity_minutes
 -- FROM gold_events_with_time_usage 
 -- WHERE efficiency_rating = 'Critical'
 --   AND event_timestamp_ts >= CURRENT_DATE - INTERVAL 7 DAYS
 -- ORDER BY lost_productivity_minutes DESC;
 --
 -- 5. Analyze truck cycle performance with truck details
 -- SELECT truck_id, truck_type, capacity_tonnes,
 --        COUNT(*) as total_cycles,
 --        AVG(payload_utilization_percentage) as avg_payload_utilization,
 --        AVG(total_cycle_duration_minutes) as avg_cycle_duration,
 --        AVG(payload_efficiency_rating) as avg_efficiency_rating
 -- FROM gold_truck_cycles 
 -- WHERE cycle_date >= CURRENT_DATE - INTERVAL 7 DAYS
 -- GROUP BY truck_id, truck_type, capacity_tonnes
 -- ORDER BY avg_payload_utilization DESC;
 --
 -- 6. Find cycles with excellent payload efficiency
 -- SELECT cycle_id, truck_id, truck_type, payload_tonnes, capacity_tonnes,
 --        payload_utilization_percentage, payload_efficiency_rating,
 --        total_cycle_duration_minutes, cycle_speed_rating
 -- FROM gold_truck_cycles 
 -- WHERE payload_efficiency_rating = 'Excellent'
 --   AND cycle_date >= CURRENT_DATE - INTERVAL 7 DAYS
 -- ORDER BY payload_utilization_percentage DESC;
 --
 -- 7. Analyze cycle performance by time of day
 -- SELECT time_of_day, day_type,
 --        COUNT(*) as total_cycles,
 --        AVG(payload_utilization_percentage) as avg_payload_utilization,
 --        AVG(total_cycle_duration_minutes) as avg_cycle_duration
 -- FROM gold_truck_cycles 
 -- WHERE cycle_date >= CURRENT_DATE - INTERVAL 7 DAYS
 -- GROUP BY time_of_day, day_type
 -- ORDER BY avg_payload_utilization DESC;
